# 目的
以下のファイルで確認してきたことを元に，ONNX Runtiem Servingを使ってモデルのServingを行い，それにリクエストして推論結果を得る．
- `load_inception_v3.ipynb`
- `save_inception_v3.ipynb`
- `enc_dec_check.ipynb`

このnotebookがクライアントという想定．
モデルの推論器サーバは別でDockerコンテナとして起動しておく．

## データの作成

In [3]:
import json
from logging import getLogger
from typing import Any, List
import numpy as np
import grpc
import joblib
import numpy as np
from PIL import Image
from pydantic import BaseModel
from proto import onnx_ml_pb2, predict_pb2, prediction_service_pb2_grpc
from torchvision import transforms
import torch

# 推論器に入れる前の前処理
preprocess = transforms.Compose([
    transforms.Resize(299),
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# 画像データへのパス
IMAGE_PATH = './data/siamese_cat.jpg'

In [4]:
# クライアント側で画像をロードして，前処理を行う
img_raw = Image.open(IMAGE_PATH)
preprocessed = preprocess(img_raw)
preprocessed = torch.unsqueeze(preprocessed, 0).numpy() # add batch dim
preprocessed.shape

(1, 3, 299, 299)

In [5]:
# ONNX形式のモデルに入力できる形にする
input_tensor = onnx_ml_pb2.TensorProto()
input_tensor.dims.extend(preprocessed.shape)
input_tensor.data_type = 1
input_tensor.raw_data = preprocessed.tobytes()

In [6]:
type(input_tensor)

onnx_ml_pb2.TensorProto

In [7]:
input_tensor.dims

[1, 3, 299, 299]

## 推論リクエストのメッセージ作成

In [8]:
# 推論器へのリクエストメッセージ作成
request_message = predict_pb2.PredictRequest()
request_message.inputs["actual_input_1"].data_type = input_tensor.data_type
request_message.inputs["actual_input_1"].dims.extend(preprocessed.shape)
request_message.inputs["actual_input_1"].raw_data = input_tensor.raw_data

In [9]:
type(request_message)


predict_pb2.PredictRequest

In [10]:
print(request_message)

inputs {
  key: "actual_input_1"
  value {
    dims: 1
    dims: 3
    dims: 299
    dims: 299
    data_type: 1
    raw_data: "\343\021\260?\004\223\223?r&\232?\341\271\240?t~\251?N\365\227?\004\223\223?\004\223\223?L\235\210?OM\247?\276\340\255?\227W\234?N\365\227?\343\021\260?\343\021\260?)\304\225?\341\271\240?+\034\245?\004\223\223?\'O{?q\316\212?t~\251?+\034\245?\341\271\240?\227W\234?\225\377\214?N\365\227?\341\271\240?\337a\221?\006\353\242?t~\251?\276\340\255?\231\257\253?,t\264?\006\353\242?\006\353\242?\276\340\255?+\034\245?N\365\227?N\365\227?\274\210\236?\004\223\223?\227W\234?\010C\262?+\034\245?\231\257\253?\233\007\273?t~\251?\006\353\242?\337a\221?OM\247?\231\257\253?,t\264?)\304\225?)\304\225?\227W\234?t~\251?\276\340\255?\231\257\253?\010C\262?OM\247?\276\340\255?+\034\245?\276\340\255?\343\021\260?\010C\262?N\365\227?N\365\227?+\034\245?\231\257\253?\276\340\255?\337a\221?r&\232?,t\264?\276\340\255?,t\264?v\326\270?\343\021\260?\010C\262?v\326\270?\010C\262?+\034\24

ここまででクライアント側のデータ準備はOK．
あとは通信関連の設定をしてリクエストを送るだけ．

## 推論器サーバの起動
サーバを起動させるには以下の2つのコマンドを打つ：

`docker build -t onnx_runtiem_serving .`

`docker run --name="myorts" -d -p 8001:8001 -p 50051:50051 onnx_runtiem_serving`

In [11]:
# 通信関連の準備
serving_address = 'localhost:50051'
channel = grpc.insecure_channel(serving_address)
stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

In [12]:
print(type(channel))
print(type(stub))

<class 'grpc._channel.Channel'>
<class 'proto.prediction_service_pb2_grpc.PredictionServiceStub'>


In [13]:
# リクエストを送ってレスポンスを得る
# 推論器サーバが起動していないとエラーになる
response = stub.Predict(request_message)

In [14]:
print(type(response))
print(type(response.outputs))
print(type(response.outputs["output1"]))
print(type(response.outputs["output1"].raw_data))

<class 'predict_pb2.PredictResponse'>
<class 'google.protobuf.pyext._message.MessageMapContainer'>
<class 'onnx_ml_pb2.TensorProto'>
<class 'bytes'>


In [15]:
# 推論結果から出力を取得
# まず予測値を取り出して，numpy配列にする
output = np.frombuffer(response.outputs["output1"].raw_data, dtype=np.float32)

In [16]:
print(type(output))

<class 'numpy.ndarray'>


In [25]:
# 予測値を確率に変換するためにsoftmaxする
probabilities = torch.nn.functional.softmax(torch.tensor(output), dim=0)

In [47]:
# 予測確率から予測ラベルを出力
# Read the categories
with open("./data/imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]
# Show top categories per image
top5_prob, top5_catid = torch.topk(probabilities, 5)
for i in range(top5_prob.size(0)):
    print(categories[top5_catid[i]], top5_prob[i].item())

Siamese cat 0.8431887030601501
lynx 0.020345453172922134
Egyptian cat 0.020162323489785194
tabby 0.0020235965494066477
wallaby 0.0017176532419398427
